<a href="https://colab.research.google.com/github/mcoupremanne/Matching_biblio/blob/main/Scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re

def prepare_inputs(publications_df, keywords_df):
    # Convertir les colonnes en minuscules
    publications_df['titre'] = publications_df['titre'].str.lower()
    publications_df['abstract'] = publications_df['abstract'].str.lower()
    publications_df['keywords'] = publications_df['keywords'].str.lower()

    keywords_df['Keyword'] = keywords_df['Keyword'].str.lower()

    # Nettoyer les données du DF  de publications
    publications_df['titre'] = publications_df['titre'].apply(lambda x: re.sub(r'\s+', ' ', x))  # Remplacer les espaces multiples par un espace unique
    publications_df['abstract'] = publications_df['abstract'].apply(lambda x: re.sub(r'\s+', ' ', x))
    publications_df['keywords'] = publications_df['keywords'].apply(lambda x: re.sub(r'\s+', ' ', x))
    publications_df['keywords'] = publications_df['keywords'].apply(lambda x: re.sub(r',+', ',', x))  # Supprimer les virgules multiples
    publications_df['keywords'] = publications_df['keywords'].apply(lambda x: x.strip(','))  # Supprimer les virgules en début ou fin de chaîne

    # Nettoyer les données du DF  de keywords dont on veut scorer les apparitions
    keywords_df['Keyword'] = keywords_df['Keyword'].str.lower().apply(lambda x: re.sub(r'\s+', ' ', x))
    keywords_df['Keyword'] = keywords_df['Keyword'].str.lower().apply(lambda x: re.sub(r',+', ',', x))
    keywords_df['Keyword'] = keywords_df['Keyword'].str.lower().apply(lambda x: x.strip(','))

    return publications_df, keywords_df



In [ ]:
def calculate_scores(publications_df, keywords_df):
    scores = []

    for index, row in publications_df.iterrows():
        title = row['titre']
        abstract = row['abstract']
        keywords = row['keywords']

        title_score = 0
        abstract_score = 0
        keywords_score = 0


        for keyword in keywords_df['Keyword']:
            if keyword in title:
                title_score += 1
            if keyword in abstract:
                abstract_score += 1
            if keyword in keywords:
                keywords_score += 1
            total_score = title_score + abstract_score + keywords_score
        scores.append({'nom': row['nom'], 'id_autheur': row['id_autheur'], 'titre': title, 'abstract': abstract, 'keywords': keywords, 'Title Score': title_score, 'Abstract Score': abstract_score, 'Keywords Score': keywords_score, 'total score': total_score})

    scores_df = pd.DataFrame(scores)
    sorted_scores_df = scores_df.sort_values(by='total score', ascending=False)  # Tri par points totaux (ordre décroissant)
    return sorted_scores_df



In [ ]:
def extract_keyword_details(publications_df, keywords_df):
    keyword_details = []

    for index, row in publications_df.iterrows():
        title = row['titre']
        abstract = row['abstract']
        keywords = row['keywords']

        for keyword in keywords_df['Keyword']:
            if keyword in title:
                keyword_details.append({'Keyword': keyword, 'Occurrence': 'title', 'Value': title, 'titre': row['titre']})
            if keyword in abstract:
                keyword_details.append({'Keyword': keyword, 'Occurrence': 'abstract', 'Value': abstract, 'titre': row['titre']})
            if keyword in keywords:
                keyword_details.append({'Keyword': keyword, 'Occurrence': 'keywords', 'Value': keywords, 'titre': row['titre']})

    keyword_details_df = pd.DataFrame(keyword_details)
    return keyword_details_df

# Essais

In [ ]:
# Essai
publications = {
    'nom': ['Max', 'Auteur 2'],
    'id_autheur': [1, 2],
    'titre': ['Décès par tracto-pelle en 2023', 'biodiversity loss et tout et tout'],
    'abstract': ['le tracto-pelle est un outil puissant...', 'Ceci est l\'abstract de la publication 2 - biodiversité'],
    'keywords': ['biodiversity , décès tracto-pelle , abeilles', 'Biodiversity Loss']
}

keywords = {
    'Keyword': ['Biodiversité', 'biodiversity,', 'tracTo-pelle', 'décès', 'keyword5']
}

publications_df = pd.DataFrame(publications)
keywords_df = pd.DataFrame(keywords)


In [ ]:
print(publications_df, keywords_df)

        nom  id_autheur                              titre  \
0       Max           1     Décès par tracto-pelle en 2023   
1  Auteur 2           2  biodiversity loss et tout et tout   

                                            abstract  \
0           le tracto-pelle est un outil puissant...   
1  Ceci est l'abstract de la publication 2 - biod...   

                                       keywords  
0  biodiversity , décès tracto-pelle , abeilles  
1                             Biodiversity Loss            Keyword
0   Biodiversité
1  biodiversity,
2   tracTo-pelle
3          décès
4       keyword5


In [ ]:
publications_df, keywords_df=prepare_inputs(publications_df, keywords_df)
print(publications_df, keywords_df)

        nom  id_autheur                              titre  \
0       Max           1     décès par tracto-pelle en 2023   
1  Auteur 2           2  biodiversity loss et tout et tout   

                                            abstract  \
0           le tracto-pelle est un outil puissant...   
1  ceci est l'abstract de la publication 2 - biod...   

                                       keywords  
0  biodiversity , décès tracto-pelle , abeilles  
1                             biodiversity loss           Keyword
0  biodiversité
1  biodiversity
2  tracto-pelle
3         décès
4      keyword5


**No more caps issues**

In [ ]:

result_df = calculate_scores(publications_df, keywords_df)
print(result_df)

        nom  id_autheur                              titre  \
0       Max           1     décès par tracto-pelle en 2023   
1  Auteur 2           2  biodiversity loss et tout et tout   

                                            abstract  \
0           le tracto-pelle est un outil puissant...   
1  ceci est l'abstract de la publication 2 - biod...   

                                       keywords  Title Score  Abstract Score  \
0  biodiversity , décès tracto-pelle , abeilles            2               1   
1                             biodiversity loss            1               1   

   Keywords Score  total score  
0               3            6  
1               1            3  


In [ ]:
keyw_df=extract_keyword_details(publications_df, keywords_df)
print(keyw_df)

        Keyword Occurrence                                              Value  \
0  biodiversity   keywords       biodiversity , décès tracto-pelle , abeilles   
1  tracto-pelle      title                     décès par tracto-pelle en 2023   
2  tracto-pelle   abstract           le tracto-pelle est un outil puissant...   
3  tracto-pelle   keywords       biodiversity , décès tracto-pelle , abeilles   
4         décès      title                     décès par tracto-pelle en 2023   
5         décès   keywords       biodiversity , décès tracto-pelle , abeilles   
6  biodiversité   abstract  ceci est l'abstract de la publication 2 - biod...   
7  biodiversity      title                  biodiversity loss et tout et tout   
8  biodiversity   keywords                                  biodiversity loss   

                               titre  
0     décès par tracto-pelle en 2023  
1     décès par tracto-pelle en 2023  
2     décès par tracto-pelle en 2023  
3     décès par tracto-pelle en 2

In [ ]:
import pandas as pd
import sqlite3

# Copie des dataframes d'origine
original_publications_df = publications_df.copy()
original_keywords_df = keywords_df.copy()

# 1. Préparer les inputs
publications_df, keywords_df = prepare_inputs(publications_df, keywords_df)

# 2. Calculer les scores
sorted_scores_df = calculate_scores(publications_df, keywords_df)

# 3. Extraire les détails des occurrences des mots-clés
keyword_details_df = extract_keyword_details(publications_df, keywords_df)

# Créer une base de données SQLite et enregistrer les dataframes en tant que tables
conn = sqlite3.connect('data.db')

# Enregistrer le dataframe original_publications_df en tant que table "original_publications"
original_publications_df.to_sql('original_publications', conn, if_exists='replace', index=False)

# Enregistrer le dataframe original_keywords_df en tant que table "original_keywords"
original_keywords_df.to_sql('original_keywords', conn, if_exists='replace', index=False)

# Enregistrer le dataframe publications_df en tant que table "publications"
publications_df.to_sql('publications', conn, if_exists='replace', index=False)

# Enregistrer le dataframe keywords_df en tant que table "keywords"
keywords_df.to_sql('keywords', conn, if_exists='replace', index=False)

# Enregistrer le dataframe sorted_scores_df en tant que table "sorted_scores"
sorted_scores_df.to_sql('sorted_scores', conn, if_exists='replace', index=False)

# Enregistrer le dataframe keyword_details_df en tant que table "keyword_details"
keyword_details_df.to_sql('keyword_details', conn, if_exists='replace', index=False)
# Créer un objet curseur
cursor = conn.cursor()
# Creer la requête pour une table de classement
query = "DROP TABLE IF EXISTS Classement;"
cursor.execute(query)
query = "create table Classement As SELECT id_autheur, nom, SUM(`Title Score`) AS `Total Title Score`, SUM(`Abstract Score`) AS `Total Abstract Score`, SUM(`Keywords Score`) AS `Total Keywords Score`, SUM(`total score`) AS `Total Score` FROM sorted_scores GROUP BY id_autheur, nom ORDER BY `Total Score` DESC;"
cursor.execute(query)

# Fermer la connexion à la base de données
conn.close()
